# Aviation Accidents Cleaning

This notebook loads the combined aviation accidents dataset and performs client-driven filtering and cleaning. It produces a cleaned CSV that will be used in the downstream analysis notebook.

In [ ]:
import pandas as pd
import numpy as np

DATA_PATH = "Combined_Aviation_With_States.csv"  # file is in the same directory as this notebook
df_raw = pd.read_csv(DATA_PATH, low_memory=False)

print("Raw shape:", df_raw.shape)
df_raw.head()

In [ ]:
# Inspect dataframe structure
df_raw.info()

## Inspect NaNs and datatypes

We inspect missingness (NaNs) and datatypes to understand what needs cleaning.

In [ ]:
# Missing values (top 25)
na_counts = df_raw.isna().sum().sort_values(ascending=False)
na_pct = (na_counts / len(df_raw)).round(4)
missing_summary = pd.DataFrame({"na_count": na_counts, "na_pct": na_pct})
missing_summary.head(25)

In [ ]:
# Datatypes
df_raw.dtypes

## Summary statistics

Numeric columns are summarized with `describe()`. For categorical columns, we use `describe(include='object')`.

In [ ]:
df_raw.describe(include=[np.number]).T

In [ ]:
df_raw.describe(include=["object"]).T.head(20)

## Filtering aircraft and events

Client constraints:

- **Aircraft.Category**: only **Airplanes**
- **Amateur.Built**: only **professional builds** (i.e., Amateur.Built == 'No')
- **Event.Date**: remove events older than **40 years** (relative to **2026-02-14** for reproducibility)

### Assumptions
- If `Aircraft.Category` is missing or not 'Airplane', we exclude it (client only wants airplanes).
- If `Amateur.Built` is missing, we treat it as **Unknown** and exclude it (client only wants professional builds).
- If `Event.Date` cannot be parsed, we exclude it (cannot verify age constraint).

In [ ]:
def clean_str_series(s: pd.Series) -> pd.Series:
    return (s.astype("string")
              .str.strip()
              .str.replace(r"\s+", " ", regex=True))

df = df_raw.copy()

# Parse dates
df["Event.Date"] = pd.to_datetime(df["Event.Date"], errors="coerce")

# Standardize relevant filter columns
df["Aircraft.Category"] = clean_str_series(df["Aircraft.Category"]).str.title()
df["Amateur.Built"] = clean_str_series(df["Amateur.Built"]).str.title().replace({"Nan": pd.NA, "": pd.NA}).fillna("Unknown")

# Client filters
cutoff = pd.Timestamp("2026-02-14") - pd.DateOffset(years=40)  # 1986-02-14
df = df[df["Aircraft.Category"].eq("Airplane")]
df = df[df["Amateur.Built"].eq("No")]
df = df[df["Event.Date"].ge(cutoff)]

print("After client filters shape:", df.shape)
df.head()

## Injuries and robustness to destruction

Client cares about:

1. **Likelihood of serious/fatal injury**
2. **Whether an aircraft was destroyed**

### Cleaning assumptions
- Injury columns are coerced to numeric (`errors='coerce'`).
- We estimate **Passengers.Est** as the sum of:
  - `Total.Fatal.Injuries` + `Total.Serious.Injuries` + `Total.Minor.Injuries` + `Total.Uninjured`
- If **all** four injury fields are missing for a row, passengers and rates are set to **NaN** (unknown).

### Derived columns
- `Passengers.Est`
- `Serious.Fatal.Injuries`
- `Fatality.Rate` = Fatal / Passengers.Est
- `SeriousOrFatal.Rate` = (Serious + Fatal) / Passengers.Est
- `Was.Destroyed` (boolean) derived from `Aircraft.damage == 'Destroyed'`

In [ ]:
inj_cols = ["Total.Fatal.Injuries","Total.Serious.Injuries","Total.Minor.Injuries","Total.Uninjured"]

for c in inj_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

all_null = df[inj_cols].isna().all(axis=1)

df["Passengers.Est"] = df[inj_cols].fillna(0).sum(axis=1).where(~all_null, np.nan)
df["Serious.Fatal.Injuries"] = (df["Total.Fatal.Injuries"].fillna(0) + df["Total.Serious.Injuries"].fillna(0)).where(~all_null, np.nan)

df["Fatality.Rate"] = np.where(df["Passengers.Est"]>0, df["Total.Fatal.Injuries"].fillna(0)/df["Passengers.Est"], np.nan)
df["SeriousOrFatal.Rate"] = np.where(df["Passengers.Est"]>0, df["Serious.Fatal.Injuries"]/df["Passengers.Est"], np.nan)

# Aircraft damage -> destroyed flag
df["Aircraft.damage"] = clean_str_series(df["Aircraft.damage"]).str.title().fillna("Unknown")
df["Was.Destroyed"] = df["Aircraft.damage"].eq("Destroyed")

df[["Total.Fatal.Injuries","Total.Serious.Injuries","Total.Minor.Injuries","Total.Uninjured",
    "Passengers.Est","Serious.Fatal.Injuries","Fatality.Rate","SeriousOrFatal.Rate",
    "Aircraft.damage","Was.Destroyed"]].head()

## Investigate the Make column

### Cleaning tasks
- Trim whitespace and normalize repeated whitespace
- Uppercase for consistent grouping (e.g., `Cessna` vs `CESSNA`)
- Remove most punctuation (keeps word characters, spaces, `&`, `/`, `-`)
- Fill missing makes with `UNKNOWN`

### Filtering threshold
For analysis, keep only makes with **>= 50 records**.

In [ ]:
# Clean Make
df["Make"] = clean_str_series(df["Make"]).str.upper().replace({"<NA>": pd.NA, "": pd.NA}).fillna("UNKNOWN")
df["Make"] = df["Make"].str.replace(r"[^\w\s&/-]", "", regex=True).str.replace(r"\s+", " ", regex=True).str.strip()

make_counts = df["Make"].value_counts()
make_counts.head(15)

In [ ]:
# Apply make threshold (>=50)
valid_makes = make_counts[make_counts >= 50].index
df = df[df["Make"].isin(valid_makes)].copy()

print("After make>=50 shape:", df.shape)
print("Unique makes:", df["Make"].nunique())

## Inspect Model column

### Tasks
- Fill missing models with `UNKNOWN_MODEL`
- Normalize whitespace and uppercase
- Check whether model labels are unique across makes
- Create derived identifier `Plane.Type = Make + ' ' + Model` to uniquely represent plane type

In [ ]:
# Clean Model
df["Model"] = clean_str_series(df["Model"]).str.upper().replace({"<NA>": pd.NA, "": pd.NA}).fillna("UNKNOWN_MODEL")
df["Model"] = df["Model"].str.replace(r"\s+", " ", regex=True).str.strip()

# Check models that appear across multiple makes
model_make_n = df.groupby("Model")["Make"].nunique().sort_values(ascending=False)
multi_make_models = model_make_n[model_make_n > 1]
print("Models appearing under multiple makes:", len(multi_make_models))
multi_make_models.head(10)

In [ ]:
# Derived unique plane type id
df["Plane.Type"] = df["Make"] + " " + df["Model"]
df[["Make","Model","Plane.Type"]].head()

## Cleaning other columns

Columns potentially related to accident outcomes:

- `Engine.Type`
- `Weather.Condition`
- `Number.of.Engines`
- `Purpose.of.flight`
- `Broad.phase.of.flight`

### Cleaning approach
- Strip whitespace and normalize casing
- Fill missing categorical values with `Unknown`
- Coerce `Number.of.Engines` to numeric (`Int64`)

In [ ]:
# Engine.Type
df["Engine.Type"] = clean_str_series(df["Engine.Type"]).str.title().fillna("Unknown")

# Weather.Condition (normalize common codes)
df["Weather.Condition"] = clean_str_series(df["Weather.Condition"]).str.upper().fillna("UNKNOWN")
df["Weather.Condition"] = df["Weather.Condition"].replace({"UNK":"UNKNOWN","UNKN":"UNKNOWN"})

# Number.of.Engines
df["Number.of.Engines"] = pd.to_numeric(df["Number.of.Engines"], errors="coerce").astype("Int64")

# Purpose / phase
df["Purpose.of.flight"] = clean_str_series(df["Purpose.of.flight"]).str.title().fillna("Unknown")
df["Broad.phase.of.flight"] = clean_str_series(df["Broad.phase.of.flight"]).str.title().fillna("Unknown")

df[["Engine.Type","Weather.Condition","Number.of.Engines","Purpose.of.flight","Broad.phase.of.flight"]].head()

## Column removal

Drop columns with too many missing values.

**Rule:** keep columns with **more than 20,000 non-null values** (evaluated at this stage after client filtering, before downstream analysis).

> Note: At this stage, we still have more than 20,000 rows, so this threshold is meaningful.

In [ ]:
non_null_counts = df.notna().sum().sort_values(ascending=False)
keep_cols = non_null_counts[non_null_counts > 20000].index.tolist()

print("Rows:", len(df))
print("Columns kept:", len(keep_cols))
non_null_counts.tail(10)

In [ ]:
df = df[keep_cols].copy()
df.shape, df.columns

## Save cleaned dataframe

We save the cleaned data to CSV for use in the analysis notebook.

In [ ]:
OUTPUT_PATH = "AviationData_Cleaned.csv"
df.to_csv(OUTPUT_PATH, index=False)
print("Saved:", OUTPUT_PATH)
df.head()